<a href="https://colab.research.google.com/github/sebinbusra/dr.chatbot/blob/main/DoctorChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
!pip install gradio faiss-cpu PyPDF2 openai==0.28 tiktoken -U langchain-community

In [4]:
import gradio as gr
import openai
import faiss
import numpy as np
import requests
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
import json


In [42]:
# OpenAI ve diğer API anahtarlarını ayarlayın
openai_api_key = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # OpenAI API anahtarınızı buraya ekleyin
openai.api_key = openai_api_key

weather_api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # OpenWeatherMap API anahtarınızı buraya ekleyin
exchange_api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # Exchangeratesapi.io API anahtarınızı buraya ekleyin


In [6]:
# PDF dosyalarının yolları
pdf_paths = ['/content/Current Essentials of Medicine.pdf'
]

# FAISS indeksi ve belgeler için global değişkenler
vector_index = None
documents = []
chat_history = []

In [8]:
# PDF'leri okuma ve indeksleme fonksiyonu
def index_pdfs():
    global vector_index, documents

    for pdf_path in pdf_paths:
        pdf_reader = PdfReader(pdf_path)
        text = ""
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
        documents.append(text)

    combined_text = " ".join(documents)

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_text(combined_text)

    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
    vectors = embeddings.embed_documents(texts)

    vector_array = np.array(vectors)

    index = faiss.IndexFlatL2(vector_array.shape[1])
    index.add(vector_array)

    vector_index = index

    print("Bilgi tabanı başarıyla oluşturuldu!")

index_pdfs()

<ipython-input-8-b3abdacc56b4>:19: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)


Bilgi tabanı başarıyla oluşturuldu!


In [9]:
# Hava durumu verilerini çekmek için fonksiyon
def fetch_weather(location):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&units=metric&lang=tr"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        temp = data['main']['temp']
        weather_description = data['weather'][0]['description']
        return f"{location} için anlık hava durumu: {temp}°C ve {weather_description}."
    else:
        return "Hava durumu bilgilerini alamadım. Lütfen konumu kontrol edip tekrar deneyin."


In [10]:
# Döviz kuru verilerini çekmek ve döviz dönüşümü yapmak için fonksiyon
def fetch_exchange_rate(base_currency, target_currency, amount=1):
    url = f"https://api.exchangeratesapi.io/v1/latest?access_key={exchange_api_key}&format=1"
    response = requests.get(url)
    data = response.json()
    rates = data.get('rates', {})
    if target_currency in rates:
        rate = rates[target_currency]
        converted_amount = float(amount) * rate
        return f"{amount} {base_currency} = {converted_amount:.2f} {target_currency}."
    else:
        return f"{target_currency} için döviz kuru bilgisi bulunamadı."


In [39]:
def format_chat_history(chat_history):
    formatted_history = ""
    for entry in chat_history:
        if entry["role"] == "user":
            formatted_history += f"<div class='chat-bubble user'>{entry['content']}</div>"
        else:
            formatted_history += f"<div class='chat-bubble assistant'>{entry['content']}</div>"

    return formatted_history

In [40]:
# GPT-4 Yanıtını oluşturmak için fonksiyon (function calling ile)
def generate_gpt4_response(prompt_input):
    global vector_index, documents, chat_history
    openai.api_key = openai_api_key

    functions = [
        {
            "name": "fetch_weather",
            "description": "Belirli bir konum için hava durumu bilgisini alır.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "Hava durumu almak istediğiniz konumun adı"
                    }
                },
                "required": ["location"],
            },
        },
        {
            "name": "fetch_exchange_rate",
            "description": "Belirli iki para birimi arasındaki döviz kuru bilgisini alır ve isteğe bağlı olarak belirli bir miktar için dönüştürme yapar.",
            "parameters": {
                "type": "object",
                "properties": {
                    "base_currency": {
                        "type": "string",
                        "description": "Döviz kurunu almak istediğiniz temel para birimi, varsayılan olarak EUR'dir."
                    },
                    "target_currency": {
                        "type": "string",
                        "description": "Döviz kurunu almak istediğiniz hedef para biriminin ISO 4217 kodu (örneğin: TRY)"
                    },
                    "amount": {
                        "type": "number",
                        "description": "Dönüştürmek istediğiniz miktar (varsayılan olarak 1)."
                    }
                },
                "required": ["base_currency", "target_currency", "amount"],
            },
        }
    ]

    # OpenAI API çağrısı (function calling ile)
    response = openai.ChatCompletion.create(
        model='gpt-4o-mini',
        messages=[
            {"role": "system",
             "content": "Sen tıp bilgileri ile donatılmış bir asistansın ve görevin tıbbi konulardaki sorulara cevap vermektir."},
            {"role": "user", "content": prompt_input}
        ],
        functions=functions,
        function_call="auto",  # Modelin fonksiyon çağrısına karar vermesine izin ver
        temperature=0.5,
        max_tokens=512
    )

    # Sohbet geçmişini güncelle
    chat_history.append({"role": "user", "content": prompt_input})

    # Bir fonksiyon çağrısı istenip istenmediğini kontrol et
    if 'choices' in response and response['choices'][0]['finish_reason'] == 'function_call':
        function_call_info = response['choices'][0]['message']['function_call']
        function_name = function_call_info['name']
        arguments = json.loads(function_call_info['arguments'])

        if function_name == 'fetch_weather':
            location = arguments['location']
            weather_response = fetch_weather(location)
            chat_history.append({"role": "assistant", "content": weather_response})
            return format_chat_history(chat_history)
        elif function_name == 'fetch_exchange_rate':
            base_currency = arguments['base_currency']
            target_currency = arguments['target_currency']
            amount = arguments.get('amount', 1)
            exchange_rate_response = fetch_exchange_rate(base_currency, target_currency, amount)
            chat_history.append({"role": "assistant", "content": exchange_rate_response})
            return format_chat_history(chat_history)

    # Asistanın yanıtını al
    assistant_response = response['choices'][0]['message']['content'].strip()
    chat_history.append({"role": "assistant", "content": assistant_response})

    return format_chat_history(chat_history)

In [41]:
with gr.Blocks() as demo:
    # CSS ile kaydırma özelliği ve stil düzenlemeleri
    custom_css = """
    /* Chat balonları için stil */
    .chat-bubble {
        padding: 10px;
        border-radius: 10px;
        margin-bottom: 10px;
        max-width: 60%;
        word-wrap: break-word;
    }

    .user {
        background-color: #d1e7dd;
        text-align: right;
        margin-left: auto;
    }

    .assistant {
        background-color: #f8d7da;
        text-align: left;
        margin-right: auto;
    }

    /* Kaydırılabilir sohbet kutusu */
    #output-box {
        height: 400px;  /* Sabit yükseklik */
        width: 100%;
        overflow-y: scroll !important;  /* Kaydırmayı gizle ama JS ile açacağız */
        padding: 10px;
        border: 1px solid #ccc;
        border-radius: 10px;
        background-color: #f8f9fa;
        margin-bottom: 20px;
    }

    /* Giriş kutusu */
    #input-box {
        height: 150px;
        width: 100%;
    }
    """

    output_textbox = gr.HTML(label="Yanıt", elem_id="output-box")  # HTML bileşeni ile sohbeti gösteriyoruz
    input_textbox = gr.Textbox(label="Sorunuzu girin", lines=4, elem_id="input-box")  # Giriş alanı

    # Sorgu gönderildiğinde çalıştırılacak fonksiyon
    def on_submit(prompt_input):
        response = generate_gpt4_response(prompt_input)
        return response, ""  # Giriş kutusunu temizle

    # Sohbeti temizlemek için fonksiyon
    def clear_chat():
        global chat_history
        chat_history.clear()
        return "", ""  # Hem giriş kutusunu hem de sohbeti temizle

    # Giriş kutusuna "Enter" basıldığında çalıştırılan fonksiyon
    input_textbox.submit(on_submit, inputs=input_textbox, outputs=[output_textbox, input_textbox])

    # Mesaj gönderme ve sohbeti temizleme butonları
    submit_btn = gr.Button("Gönder")
    clear_btn = gr.Button("Chat'i Temizle")

    submit_btn.click(on_submit, inputs=input_textbox, outputs=[output_textbox, input_textbox])
    clear_btn.click(clear_chat, outputs=[output_textbox, input_textbox])

    demo.css = custom_css
    demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://1046f76164fb165e08.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b162ad70725dccd76c.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://2eae138fb007aae19d.gradio.live
Killing tunnel 127.0.0.1:7862 <> https://1dcffbd5ed4db3ce8d.gradio.live
Killing tunnel 127.0.0.1:7863 <> https://c9b054571b98902445.gradio.live
Killing tunnel 127.0.0.1:7864 <> https://6311d0f05878886aba.gradio.live
Killing tunnel 127.0.0.1:7865 <> https://44bc8391d8b84b0e1d.gradio.live
Killing tunnel 127.0.0.1:7866 <> https://1046f76164fb165e08.gradio.live
